<a href="https://colab.research.google.com/github/Yash-200452231/llm-codechallenge/blob/main/Edgecom_Energy.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Coding Challenge Edgecom Energy

## Installing libraries

In [1]:
!pip install -q transformers[torch] datasets accelerate peft bitsandbytes trl

## Sample Dataset

In [2]:
import pandas as pd

# Define the dataset
data = {
    'Text_Sequence': [
        "add a constraint of 4X + 5Y <= 40 in our model.",
        "please include the constraint 7X - 3Y == 15 in our Pyomo model.",
        "could you add 5X + 8Y >= 60 as a new constraint?",
        "I need a constraint: 9X + 2Y < 25 added to the model.",
        "add 3X + 4Y > 18 as a constraint.",
        "please add a constraint of 6X + 7Y <= 50.",
        "we need to include 8X - 2Y >= 10 in the constraints.",
        "let's add 2X + 9Y == 35 as another constraint.",
        "can you include the constraint 4X - 6Y < 20 in the model?",
        "add a new constraint: 3X + 5Y > 28.",
        "let's include a constraint of 7X + 3Y <= 45.",
        "add 5X - 2Y >= 20 to the constraints.",
        "I need the constraint 6X + 4Y == 50 added.",
        "please add 2X - 3Y < 15 as a constraint.",
        "add the constraint 9X + 7Y > 70 to the model.",
        "include 4X + 6Y <= 60 as a new constraint.",
        "can you add 3X - 5Y >= 25 in the model constraints?",
        "add a constraint 5X + 4Y == 40 to the model.",
        "please include 6X - 2Y < 22 in the constraints.",
        "add another constraint: 8X + 5Y > 55."
    ],
    'Label': [
        "model.constraint3 = Constraint(expr=4 * model.x + 5 * model.y <= 40)",
        "model.constraint3 = Constraint(expr=7 * model.x - 3 * model.y == 15)",
        "model.constraint3 = Constraint(expr=5 * model.x + 8 * model.y >= 60)",
        "model.constraint3 = Constraint(expr=9 * model.x + 2 * model.y < 25)",
        "model.constraint3 = Constraint(expr=3 * model.x + 4 * model.y > 18)",
        "model.constraint3 = Constraint(expr=6 * model.x + 7 * model.y <= 50)",
        "model.constraint3 = Constraint(expr=8 * model.x - 2 * model.y >= 10)",
        "model.constraint3 = Constraint(expr=2 * model.x + 9 * model.y == 35)",
        "model.constraint3 = Constraint(expr=4 * model.x - 6 * model.y < 20)",
        "model.constraint3 = Constraint(expr=3 * model.x + 5 * model.y > 28)",
        "model.constraint3 = Constraint(expr=7 * model.x + 3 * model.y <= 45)",
        "model.constraint3 = Constraint(expr=5 * model.x - 2 * model.y >= 20)",
        "model.constraint3 = Constraint(expr=6 * model.x + 4 * model.y == 50)",
        "model.constraint3 = Constraint(expr=2 * model.x - 3 * model.y < 15)",
        "model.constraint3 = Constraint(expr=9 * model.x + 7 * model.y > 70)",
        "model.constraint3 = Constraint(expr=4 * model.x + 6 * model.y <= 60)",
        "model.constraint3 = Constraint(expr=3 * model.x - 5 * model.y >= 25)",
        "model.constraint3 = Constraint(expr=5 * model.x + 4 * model.y == 40)",
        "model.constraint3 = Constraint(expr=6 * model.x - 2 * model.y < 22)",
        "model.constraint3 = Constraint(expr=8 * model.x + 5 * model.y > 55)"
    ]
}


In [3]:
len(data['Label']), len(data['Text_Sequence'])

(20, 20)

In [4]:
from datasets import Dataset

# Create a Dataset object
dataset = Dataset.from_dict(data)

# Display the dataset
dataset

Dataset({
    features: ['Text_Sequence', 'Label'],
    num_rows: 20
})

## Basic Training script for T5

In [4]:
from transformers import T5Tokenizer, T5ForConditionalGeneration, Trainer, TrainingArguments
import pandas as pd
import torch


# Create DataFrame
df = pd.DataFrame(data)

# Initialize the tokenizer and model
model_name = 'google/flan-t5-small'
tokenizer = T5Tokenizer.from_pretrained(model_name)
model = T5ForConditionalGeneration.from_pretrained(model_name)

# Prepare the dataset
train_texts = df['Text_Sequence'].tolist()
train_labels = df['Label'].tolist()

# Tokenize the dataset
train_encodings = tokenizer(train_texts, truncation=True, padding='max_length', max_length=512, return_tensors='pt')
train_labels_encodings = tokenizer(train_labels, truncation=True, padding='max_length', max_length=512, return_tensors='pt')

# Define Dataset Class
class ConstraintDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: val[idx] for key, val in self.encodings.items()}
        item['labels'] = self.labels['input_ids'][idx]
        return item

    def __len__(self):
        return len(self.encodings.input_ids)

train_dataset = ConstraintDataset(train_encodings, train_labels_encodings)

# Define training arguments
training_args = TrainingArguments(
    output_dir='./results',
    num_train_epochs=3,
    per_device_train_batch_size=4,
    save_steps=10_000,
    save_total_limit=2,
)

# Initialize the Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
)

# Fine-tune the model
trainer.train()


You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


Step,Training Loss


TrainOutput(global_step=15, training_loss=39.9119384765625, metrics={'train_runtime': 339.4107, 'train_samples_per_second': 0.177, 'train_steps_per_second': 0.044, 'total_flos': 11153430282240.0, 'train_loss': 39.9119384765625, 'epoch': 3.0})

In [5]:
from transformers import T5Tokenizer, T5ForConditionalGeneration

# Load the fine-tuned model and tokenizer
#model_path = './results'  # Path to the fine-tuned model directory
#tokenizer = T5Tokenizer.from_pretrained(model_path)
#model = T5ForConditionalGeneration.from_pretrained(model_path)

# Function to generate constraint code from input text
def generate_constraint_code(input_text):
    # Tokenize the input text
    input_ids = tokenizer(input_text, return_tensors='pt').input_ids

    # Generate the output using the model
    outputs = model.generate(input_ids)

    # Decode the generated output
    generated_code = tokenizer.decode(outputs[0], skip_special_tokens=True)
    return generated_code

# Example input text sequence
input_text = "add a constraint of 2X + 6Y > 30 in our model."

# Generate and print the output
generated_code = generate_constraint_code(input_text)
print(f"Input: {input_text}")
print(f"Generated Code: {generated_code}")


/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1168: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


Input: add a constraint of 2X + 6Y > 30 in our model.
Generated Code: Add a constraint of 2X + 6Y > 30 in our model.


## PEFT(LoRA) Training with quantization

### Dataset for peft


In [38]:
data_llama = {}

train_texts = data['Text_Sequence']
train_labels = data['Label']

data_llama['Text_Sequence'] = [ "USER### "+x+"CODE### "+y+"[EOS]" for x,y in zip(train_texts, train_labels) ]

#data_llama
train_dataset = Dataset.from_dict(data_llama)
train_dataset

Dataset({
    features: ['Text_Sequence'],
    num_rows: 20
})

In [39]:
train_dataset['Text_Sequence']

['USER### add a constraint of 4X + 5Y <= 40 in our model.CODE### model.constraint3 = Constraint(expr=4 * model.x + 5 * model.y <= 40)[EOS]',
 'USER### please include the constraint 7X - 3Y == 15 in our Pyomo model.CODE### model.constraint3 = Constraint(expr=7 * model.x - 3 * model.y == 15)[EOS]',
 'USER### could you add 5X + 8Y >= 60 as a new constraint?CODE### model.constraint3 = Constraint(expr=5 * model.x + 8 * model.y >= 60)[EOS]',
 'USER### I need a constraint: 9X + 2Y < 25 added to the model.CODE### model.constraint3 = Constraint(expr=9 * model.x + 2 * model.y < 25)[EOS]',
 'USER### add 3X + 4Y > 18 as a constraint.CODE### model.constraint3 = Constraint(expr=3 * model.x + 4 * model.y > 18)[EOS]',
 'USER### please add a constraint of 6X + 7Y <= 50.CODE### model.constraint3 = Constraint(expr=6 * model.x + 7 * model.y <= 50)[EOS]',
 'USER### we need to include 8X - 2Y >= 10 in the constraints.CODE### model.constraint3 = Constraint(expr=8 * model.x - 2 * model.y >= 10)[EOS]',
 "USER#

### Peft Codellama Python variation

In [12]:
from huggingface_hub import notebook_login

notebook_login()

In [13]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig
from peft import LoraConfig

# Load the 7b llama model
model_id = "meta-llama/CodeLlama-7b-Python-hf"

quantization_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_compute_dtype=torch.float16,
    bnb_4bit_quant_type="nf4"
)

# Load model
model = AutoModelForCausalLM.from_pretrained(model_id, quantization_config=quantization_config)

# Load tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_id, trust_remote_code=True)
# Set it to a new token to correctly attend to EOS tokens.
tokenizer.add_special_tokens({'pad_token': '<PAD>'})

`low_cpu_mem_usage` was None, now set to True since model is quantized.


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

1

In [14]:
lora_config = LoraConfig(
    r=8,
    target_modules=["q_proj", "o_proj", "k_proj", "v_proj", "gate_proj", "up_proj", "down_proj"],
    bias="none",
    task_type="CAUSAL_LM",
)

model.add_adapter(lora_config)

In [16]:
from transformers import TrainingArguments

YOUR_HF_USERNAME = ""

output_dir = f"/content/llama-7b-qlora-ultrachat"
per_device_train_batch_size = 4
gradient_accumulation_steps = 4
optim = "paged_adamw_32bit"
save_steps = 10
logging_steps = 10
learning_rate = 2e-4
max_grad_norm = 0.3
max_steps = 1000
num_train_epochs = 1
warmup_ratio = 0.03
lr_scheduler_type = "constant"

training_arguments = TrainingArguments(
    output_dir=output_dir,
    per_device_train_batch_size=per_device_train_batch_size,
    gradient_accumulation_steps=gradient_accumulation_steps,
    optim=optim,
    save_steps=save_steps,
    logging_steps=logging_steps,
    learning_rate=learning_rate,
    max_grad_norm=max_grad_norm,
    num_train_epochs=num_train_epochs,
    #max_steps=max_steps,
    warmup_ratio=warmup_ratio,
    lr_scheduler_type=lr_scheduler_type,
    gradient_checkpointing=True,
    push_to_hub=False,
)

In [31]:
from trl import SFTTrainer

trainer = SFTTrainer(
    model=model,
    args=training_arguments,
    train_dataset=train_dataset,
    packing=True,
    dataset_text_field="Text_Sequence",
    tokenizer=tokenizer,
    max_seq_length=128,
    num_of_sequences=len(train_dataset["Text_Sequence"]),
    dataset_batch_size=4,
    #formatting_func=formatting_func
)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_deprecation.py:100: FutureWarning: Deprecated argument(s) used in '__init__': packing, dataset_text_field, max_seq_length, num_of_sequences, dataset_batch_size. Will not be supported from version '1.0.0'.

Deprecated positional argument(s) used in SFTTrainer, please use the SFTConfig to set these arguments instead.
  warnings.warn(message, FutureWarning)
/usr/local/lib/python3.10/dist-packages/trl/trainer/sft_trainer.py:192: UserWarning: You passed a `packing` argument to the SFTTrainer, the value you passed will override the one in the `SFTConfig`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/trl/trainer/sft_trainer.py:280: UserWarning: You passed a `max_seq_length` argument to the SFTTrainer, the value you passed will override the one in the `SFTConfig`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/trl/trainer/sft_trainer.py:301: UserWarning: You passed a `dataset_batch_size` argument to the SFTTra

Generating train split: 0 examples [00:00, ? examples/s]

/usr/local/lib/python3.10/dist-packages/trl/trainer/sft_trainer.py:408: UserWarning: You passed a tokenizer with `padding_side` not equal to `right` to the SFTTrainer. This might lead to some unexpected behaviour due to overflow issues when training a model in half-precision. You might consider adding `tokenizer.padding_side = 'right'` to your code.
  warnings.warn(


In [40]:
trainer.train()

/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:464: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.4 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  warnings.warn(


Step,Training Loss


TrainOutput(global_step=1, training_loss=1.2300381660461426, metrics={'train_runtime': 3.9267, 'train_samples_per_second': 2.037, 'train_steps_per_second': 0.255, 'total_flos': 40718328397824.0, 'train_loss': 1.2300381660461426, 'epoch': 1.0})

### Testing the model

In [41]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig

#tokenizer = AutoTokenizer.from_pretrained(model_id)

#quantization_config = BitsAndBytesConfig(load_in_4bit=True, bnb_4bit_compute_dtype=torch.float16)

#model = AutoModelForCausalLM.from_pretrained(
#    model_id,
#    quantization_config=quantization_config
#)

text = "USER### add a constraint of 2X + 6Y > 30 in our model.CODE### "
inputs = tokenizer(text, return_tensors="pt")
outputs = model.generate(inputs["input_ids"], max_new_tokens=128, do_sample=False)

print("After attaching Lora Adapters:")
print(tokenizer.decode(outputs[0], skip_special_tokens=False))

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1659: UserWarning: You are calling .generate() with the `input_ids` being on a device type different than your model's device. `input_ids` is on cpu, whereas the model is on cuda. You may experience unexpected behaviors or slower generation. Please make sure that you have put `input_ids` to the correct device by calling for example input_ids = input_ids.to('cuda') before running `.generate()`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:91: UserWarning: None of the inputs have requires_grad=True. Gradients will be None
  warnings.warn(


After attaching Lora Adapters:
<s> USER### add a constraint of 2X + 6Y > 30 in our model.CODE### 

# add a constraint of 2X + 6Y > 30 in our model.
model.add_constraint(2*x + 6*y >= 30)

# solve the model
model.solve()

# print the results
print("x:", x.solution_value)
print("y:", y.solution_value)

# print the objective value
print("objective:", model.objective.value())

# print the solution
print("solution:")
for i in range(len(
